# CoDeepNEAT demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using CoDeepNEAT to solve CIFAR-10

In [1]:
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from math import pi, floor
from random import random
from codeepneat import codeepneat, config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras

Using TensorFlow backend.


### Problem: CIFAR10 data set

Conveniently, it's also built into Keras, which our CoDeepNEAT imiplementation is built off of.

In [2]:
(x_train_all, y_train_all), (x_test, y_test) = cifar10.load_data()
y_train_all = y_train_all[:,0]
y_test = y_test[:,0]

x_train_all = np.reshape(x_train_all, (x_train_all.shape[0], 32, 32, 3)).astype('float32') / 255
x_test = np.reshape(x_test, (x_test.shape[0], 32, 32, 3)).astype('float32') / 255
y_train_all = keras.utils.np_utils.to_categorical(y_train_all)
y_test = keras.utils.np_utils.to_categorical(y_test)

index = np.array(range(len(x_train_all)))
np.random.shuffle(index)
index_train = index[:42500]
index_val = index[42500:]
x_train = x_train_all[index_train]
y_train = y_train_all[index_train]
x_val = x_train_all[index_val]
y_val = y_train_all[index_val]

data = [x_train, y_train, x_val, y_val, x_test, y_test]
print("data shapes")
print("  x train:", x_train.shape)
print("  y train:", y_train.shape)

print("  x val:", x_val.shape)
print("  y val:", y_val.shape)

print("  x test:", x_test.shape)
print("  y test:", y_test.shape)

data shapes
  x train: (42500, 32, 32, 3)
  y train: (42500, 10)
  x val: (7500, 32, 32, 3)
  y val: (7500, 10)
  x test: (10000, 32, 32, 3)
  y test: (10000, 10)


### Configuring NEAT

Many of the options and inputs are still handled through the config file. The config file has been shortened considerably as many parameters have been eliminated, although many parameters have also been introduced which could be added.

In [7]:
%%file configCIFAR10
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 32,32,3
output_nodes        = 10
conv                = True
LSTM                = False

[genetic]
max_fitness_threshold = 1

# Human reasoning
pop_size              = 10
prob_addconv          = 0.9
prob_addLSTM          = 0.0
prob_addlayer         = 0.1
prob_addmodule        = 0.05
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coefficient      = 5.0
disjoint_coefficient    = 3.0
connection_coefficient  = 0.4
size_coefficient        = 0.8

[species]
species_size        = 10
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configCIFAR10


### Fitness

For this demonstration we'll be using supervised learning to train the networks produced by CoDeepNEAT on CIFAR-10 and will use their accuracy after 5 epochs as our fitness. CIFAR-10, like MNIST, is a 10 category classification problem.

In [8]:
def fitness(network, data):
    network.fit(data[0], data[1],  epochs=5)
    loss, acc = network.evaluate(data[2], data[3])
    return acc

### Evolution

Evolution with CoDeepNEAT is slightly different than evolution with NEAT. The main difference is coevolution, where we have two separate populations with a hierarchical relationship evolving together.

In [9]:
def evolve(n, debugging=False):
    if(debugging):
        debug = open("debug.txt", "w")
    else:
        debug = None
    config.load('configCIFAR10')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(15, chromosome.ModuleChromo, debug=debug)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(10, chromosome.BlueprintChromo, module_pop, debug=debug)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, 25, fitness, data, save_best=True, name='CIFAR10', debug=debug)
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="CIFAR10mod_")
    visualize.plot_stats(blueprint_pop.stats, name="CIFAR10bp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [ ]:
evolve(25, True)

-----Generation 0--------
Network 0
Epoch 1/5
42500/42500 [==============================] - 30s - loss: 1.6410 - acc: 0.4255    
Epoch 2/5
42500/42500 [==============================] - 30s - loss: 1.4733 - acc: 0.4896    
Epoch 3/5
42500/42500 [==============================] - 30s - loss: 1.4179 - acc: 0.5123    
Epoch 4/5
42500/42500 [==============================] - 30s - loss: 1.3729 - acc: 0.5314    
Epoch 5/5
7500/7500 [==============================] - 2s     

Network 0 Fitness: 0.492133333365
Network 1
Epoch 1/5
42500/42500 [==============================] - 22s - loss: 1.7191 - acc: 0.3920    
Epoch 2/5
42500/42500 [==============================] - 21s - loss: 1.5441 - acc: 0.4588    
Epoch 3/5
42500/42500 [==============================] - 22s - loss: 1.4784 - acc: 0.4844    
Epoch 4/5
42500/42500 [==============================] - 21s - loss: 1.4385 - acc: 0.5014    
Epoch 5/5
7232/7500 [===========================>..] - ETA: 0s
Network 1 Fitness: 0.485200000064
Network

42500/42500 [==============================] - 30s - loss: 1.4757 - acc: 0.4898    
Epoch 3/5
42500/42500 [==============================] - 30s - loss: 1.4143 - acc: 0.5141    
Epoch 4/5
42500/42500 [==============================] - 30s - loss: 1.3732 - acc: 0.5293     - ETA: 
Epoch 5/5
7456/7500 [============================>.] - ETA: 0s
Network 14 Fitness: 0.504133333365
Network 15
Epoch 1/5
42500/42500 [==============================] - 25s - loss: 1.6603 - acc: 0.4178    
Epoch 2/5
42500/42500 [==============================] - 25s - loss: 1.4845 - acc: 0.4829    
Epoch 3/5
42500/42500 [==============================] - 25s - loss: 1.4264 - acc: 0.5071    
Epoch 4/5
42500/42500 [==============================] - 25s - loss: 1.3878 - acc: 0.5216    
Epoch 5/5
7456/7500 [============================>.] - ETA: 0s
Network 15 Fitness: 0.474666666698
Network 16
Epoch 1/5
42500/42500 [==============================] - 25s - loss: 1.6381 - acc: 0.4258    
Epoch 2/5
42500/42500 [=========

42500/42500 [==============================] - 11s - loss: 1.9078 - acc: 0.3059    
Epoch 2/5
42500/42500 [==============================] - 11s - loss: 1.7336 - acc: 0.3801    
Epoch 3/5
42500/42500 [==============================] - 11s - loss: 1.6670 - acc: 0.4051    
Epoch 4/5
42500/42500 [==============================] - 11s - loss: 1.6236 - acc: 0.4236    
Epoch 5/5
7424/7500 [============================>.] - ETA: 0s
Network 2 Fitness: 0.397733333333
Network 3
Epoch 1/5
42500/42500 [==============================] - 30s - loss: 1.6464 - acc: 0.4250    
Epoch 2/5
42500/42500 [==============================] - 30s - loss: 1.4665 - acc: 0.4949    
Epoch 3/5
42500/42500 [==============================] - 30s - loss: 1.4090 - acc: 0.5196    
Epoch 4/5
42500/42500 [==============================] - 30s - loss: 1.3642 - acc: 0.5341    
Epoch 5/5
7456/7500 [============================>.] - ETA: 0s
Network 3 Fitness: 0.499333333333
Network 4
Epoch 1/5
42500/42500 [=====================

42500/42500 [==============================] - 4s - loss: 1.6670 - acc: 0.4054     
Epoch 4/5
42500/42500 [==============================] - 4s - loss: 1.6207 - acc: 0.4233     
Epoch 5/5
6912/7500 [==========================>...] - ETA: 0s
Network 16 Fitness: 0.372533333365
Network 17
Epoch 1/5
42500/42500 [==============================] - 7s - loss: 1.9279 - acc: 0.3000     
Epoch 2/5
42500/42500 [==============================] - 6s - loss: 1.7554 - acc: 0.3753     
Epoch 3/5
42500/42500 [==============================] - 6s - loss: 1.6917 - acc: 0.4014     
Epoch 4/5
42500/42500 [==============================] - 6s - loss: 1.6509 - acc: 0.4144     
Epoch 5/5
7136/7500 [===========================>..] - ETA: 0s
Network 17 Fitness: 0.410933333397
Network 18
Epoch 1/5
42500/42500 [==============================] - 11s - loss: 1.9125 - acc: 0.3079    
Epoch 2/5
42500/42500 [==============================] - 11s - loss: 1.7471 - acc: 0.3693    
Epoch 3/5
42500/42500 [=================

42500/42500 [==============================] - 16s - loss: 1.5997 - acc: 0.4386    
Epoch 3/5
42500/42500 [==============================] - 16s - loss: 1.5343 - acc: 0.4633    
Epoch 4/5
42500/42500 [==============================] - 16s - loss: 1.5047 - acc: 0.4768    
Epoch 5/5
7424/7500 [============================>.] - ETA: 0s
Network 4 Fitness: 0.48226666673
Network 5
Epoch 1/5
42500/42500 [==============================] - 31s - loss: 1.6536 - acc: 0.4217    
Epoch 2/5
42500/42500 [==============================] - 30s - loss: 1.4748 - acc: 0.4901    
Epoch 3/5
42500/42500 [==============================] - 30s - loss: 1.4143 - acc: 0.5152    
Epoch 4/5
42500/42500 [==============================] - 30s - loss: 1.3703 - acc: 0.5318    
Epoch 5/5
7500/7500 [==============================] - 4s     

Network 5 Fitness: 0.494666666698
Network 6
Epoch 1/5
42500/42500 [==============================] - 24s - loss: 1.9948 - acc: 0.2869    
Epoch 2/5
42500/42500 [=====================

...

7360/7500 [============================>.] - ETA: 0s
Network 11 Fitness: 0.275066666683
Network 12
Epoch 1/5
42500/42500 [==============================] - 7s - loss: 1.9383 - acc: 0.3009     
Epoch 2/5
42500/42500 [==============================] - 5s - loss: 1.7497 - acc: 0.3739     
Epoch 3/5
42500/42500 [==============================] - 5s - loss: 1.6799 - acc: 0.4025     
Epoch 4/5
42500/42500 [==============================] - 4s - loss: 1.6308 - acc: 0.4188     
Epoch 5/5
7392/7500 [============================>.] - ETA: 0s
Network 12 Fitness: 0.320133333349
Network 13
Epoch 1/5
42500/42500 [==============================] - 7s - loss: 1.9368 - acc: 0.2888     
Epoch 2/5
42500/42500 [==============================] - 6s - loss: 1.7674 - acc: 0.3630     
Epoch 3/5
42500/42500 [==============================] - 6s - loss: 1.6944 - acc: 0.3915     
Epoch 4/5
42500/42500 [==============================] - 6s - loss: 1.6442 - acc: 0.4099     
Epoch 5/5
7008/7500 [===================

-----Generation 7--------
Network 0
Epoch 1/5
42500/42500 [==============================] - 31s - loss: 1.8205 - acc: 0.3251    - ETA: 6s - loss: 1. - ETA: 4s
Epoch 2/5
42500/42500 [==============================] - 29s - loss: 1.6033 - acc: 0.4184    - ETA: 4s - loss: 1.6079 -  - ETA: 3s - loss:  - ETA: 2s - loss: 1.60 - ETA: 1s - loss: 1
Epoch 3/5
42500/42500 [==============================] - 29s - loss: 1.5325 - acc: 0.4468    - ETA: 1s - loss: 1.5340 - a - ETA: 1s - loss: 
Epoch 4/5
42500/42500 [==============================] - 29s - loss: 1.4835 - acc: 0.4699    
Epoch 5/5
7488/7500 [============================>.] - ETA: 0s
Network 0 Fitness: 0.4376
Network 1
Epoch 1/5
42500/42500 [==============================] - 27s - loss: 1.6809 - acc: 0.4036    
Epoch 2/5
42500/42500 [==============================] - 25s - loss: 1.3791 - acc: 0.5131    
Epoch 3/5
42500/42500 [==============================] - 25s - loss: 1.2746 - acc: 0.5512    
Epoch 4/5
42500/42500 [==================

42500/42500 [==============================] - 7s - loss: 2.0398 - acc: 0.2215     
Epoch 5/5
7360/7500 [============================>.] - ETA: 0s
Network 13 Fitness: 0.197466666667
Network 14
Epoch 1/5
42500/42500 [==============================] - 11s - loss: 1.9810 - acc: 0.2661    
Epoch 2/5
42500/42500 [==============================] - 9s - loss: 1.7891 - acc: 0.3534     
Epoch 3/5
42500/42500 [==============================] - 9s - loss: 1.7167 - acc: 0.3819     
Epoch 4/5
42500/42500 [==============================] - 9s - loss: 1.6702 - acc: 0.4006     
Epoch 5/5
7296/7500 [============================>.] - ETA: 0s
Network 14 Fitness: 0.389866666698
Network 15
Epoch 1/5
42500/42500 [==============================] - 59s - loss: 1.8752 - acc: 0.3162    
Epoch 2/5
42500/42500 [==============================] - 57s - loss: 1.6237 - acc: 0.4169    
Epoch 3/5
42500/42500 [==============================] - 57s - loss: 1.5644 - acc: 0.4387    
Epoch 4/5
42500/42500 [=================

42500/42500 [==============================] - 33s - loss: 1.3698 - acc: 0.5161    
Epoch 3/5
42500/42500 [==============================] - 33s - loss: 1.2351 - acc: 0.5634    
Epoch 4/5
42500/42500 [==============================] - 33s - loss: 1.1584 - acc: 0.5932    
Epoch 5/5
7488/7500 [============================>.] - ETA: 0s
Network 1 Fitness: 0.576266666667
Network 2
Epoch 1/5
42500/42500 [==============================] - 12s - loss: 1.9878 - acc: 0.2634    
Epoch 2/5
42500/42500 [==============================] - 9s - loss: 1.7886 - acc: 0.3527     
Epoch 3/5
42500/42500 [==============================] - 9s - loss: 1.7199 - acc: 0.3830     
Epoch 4/5
42500/42500 [==============================] - 9s - loss: 1.6754 - acc: 0.3992     
Epoch 5/5
7104/7500 [===========================>..] - ETA: 0s
Network 2 Fitness: 0.393866666698
Network 3
Epoch 1/5
42500/42500 [==============================] - 65s - loss: 2.3076 - acc: 0.0978    
Epoch 2/5
42500/42500 [=====================

42500/42500 [==============================] - 10s - loss: 1.6757 - acc: 0.3972    
Epoch 5/5
7488/7500 [============================>.] - ETA: 0s
Network 15 Fitness: 0.4008
Network 16
Epoch 1/5
42500/42500 [==============================] - 86s - loss: 1.7884 - acc: 0.3282    
Epoch 2/5
42500/42500 [==============================] - 83s - loss: 1.4625 - acc: 0.4657    
Epoch 3/5
42500/42500 [==============================] - 83s - loss: 1.3473 - acc: 0.5103    
Epoch 4/5
42500/42500 [==============================] - 83s - loss: 1.2717 - acc: 0.5395    
Epoch 5/5
7500/7500 [==============================] - 9s     

Network 16 Fitness: 0.459333333365
Network 17
Epoch 1/5
42500/42500 [==============================] - 12s - loss: 1.9798 - acc: 0.2637    
Epoch 2/5
42500/42500 [==============================] - 10s - loss: 1.7982 - acc: 0.3497    
Epoch 3/5
42500/42500 [==============================] - 9s - loss: 1.7266 - acc: 0.3790     
Epoch 4/5
42500/42500 [========================

In [ ]:
def eval_best(model_file):
    config.load('configCIFAR10')
    model = keras.models.load_model(model_file)
    visualize.draw_net(model, "_" + model_file)    
    model.fit(x_train_all, y_train_all, epochs=50)
    loss, fitness = model.evaluate(x_test, y_test)
    print("fitness", fitness)

In [ ]:
eval_best("CIFAR10_best_model_0")